### Info on data from Kaggle and UNIC
+ Compeititon's goal was to predict item sales at stores in various locations for two 28-day time periods.
+ Data is focused on series that display intermittency, i.e., sporadic demand including zeros.
+ Based on hierarchical sales data, generously made available by Walmart, starting at the item level and aggregating to that of departments, product categories and stores in three geographical areas of the US: California, Texas, and Wisconsin.
+ Besides the time series data, it also included explanatory variables such as price, promotions, day of the week, and special events (e.g. Super Bowl, Valentine’s Day, and Orthodox Easter) that affect sales which are used to improve forecasting accuracy.
+ calendar.csv - Contains information about the dates on which the products are sold.
+ sales_train_validation.csv - Contains the historical daily unit sales data per product and store [d_1 - d_1913]
+ sell_prices.csv - Contains information about the price of the products sold per store and date.
+ sales_train_evaluation.csv - Includes sales [d_1 - d_1941]

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# where the files are strored
DATA_RAW = '../data/raw/m5-forecasting-accuracy/'
DATA_PROCESSED = '../data/processed/'

sales_file = os.path.join(DATA_RAW, "sales_train_validation.csv")
calendar_file = os.path.join(DATA_RAW, "calendar.csv")
prices_file = os.path.join(DATA_RAW, "sell_prices.csv")

In [3]:
# load smaller support data
calendar = pd.read_csv(calendar_file)

# below helps with merging calendar in cells below
calendar['d'] = calendar['d'].astype(str)

prices = pd.read_csv(prices_file)

In [4]:
# need to process data in chunks. Getting memory issues
chunk_size = 10000 # num of rows to process
output_chunks = []

reader = pd.read_csv(sales_file, chunksize=chunk_size)

In [5]:
for i, chunk in enumerate(reader):
    print(f"Processing chunk {i + 1}")

    # Melt the wide format into long format
    chunk_long = pd.melt(
        chunk,
        id_vars=['id', 'item_id', 'dept_id', 'store_id', 'cat_id', 'state_id'],
        var_name='d',
        value_name='sales'
    )

    # Merge calendar info
    chunk_merged = chunk_long.merge(calendar, on="d", how="left")

    # Sort before merging prices so ffill works properly
    chunk_merged = chunk_merged.sort_values(by=['store_id', 'item_id', 'wm_yr_wk'])

    # Merge prices on store_id and item_id only so we're not joining on missing values
    chunk_merged = chunk_merged.merge(
        prices[['store_id', 'item_id', 'wm_yr_wk', 'sell_price']],
        on=['store_id', 'item_id', 'wm_yr_wk'],
        how='left'
    )

    # Forward fill sell_price within each store-item group
    chunk_merged['sell_price'] = (
        chunk_merged
        .groupby(['store_id', 'item_id'])['sell_price']
        .ffill()
    )

    # Drop rows still missing prices or date
    chunk_merged.dropna(subset=['sell_price', 'date'], inplace=True)

    print(f"Chunk {i + 1} processed. Shape: {chunk_merged.shape}")

    output_chunks.append(chunk_merged)

Processing chunk 1
Chunk 1 processed. Shape: (14885452, 22)
Processing chunk 2
Chunk 2 processed. Shape: (15394464, 22)
Processing chunk 3
Chunk 3 processed. Shape: (15008799, 22)
Processing chunk 4
Chunk 4 processed. Shape: (739242, 22)


In [6]:
# combine all chunks
full_data = pd.concat(output_chunks, ignore_index=True)

In [7]:
# check to make sure full data is there
print(full_data.shape)

(46027957, 22)


In [8]:
full_data['month'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [9]:
print(full_data['date'].min(), full_data['date'].max())

2011-01-29 2016-04-24


In [10]:
# save processed data
os.makedirs(DATA_PROCESSED, exist_ok=True)
output_path = os.path.join(DATA_PROCESSED, "sales_merged.csv")
full_data.to_csv(output_path, index=False)

print("saved data to: ", output_path)

saved data to:  ../data/processed/sales_merged.csv


In [11]:
full_data.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,CA_1,FOODS,CA,d_1,3,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,2.0
1,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,CA_1,FOODS,CA,d_2,0,2011-01-30,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,2.0
2,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,CA_1,FOODS,CA,d_3,0,2011-01-31,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,2.0
3,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,CA_1,FOODS,CA,d_4,1,2011-02-01,11101,...,2,2011,NaN,NaN,NaN,NaN,1,1,0,2.0
4,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,CA_1,FOODS,CA,d_5,4,2011-02-02,11101,...,2,2011,NaN,NaN,NaN,NaN,1,0,1,2.0
